In [1]:
# The code was removed by Watson Studio for sharing.

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:

body = client_4feb602bea494115beaacee8603bb1e0.get_object(Bucket='recommendationengine1-donotdelete-pr-pvtfgqiao8fgmh',Key='ratings.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

user_ratings = pd.read_csv(body)
user_ratings.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movie_info = movie_info.drop(['genres'], axis=1)

In [4]:
user_ratings = user_ratings.drop(['timestamp'], axis=1)

In [5]:
movie_info.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [6]:
user_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
movie_info.isnull().sum()

movieId    0
title      0
dtype: int64

In [8]:
user_ratings.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [9]:
movie_user_ratings = pd.merge(movie_info, user_ratings, on="movieId")

In [10]:
movie_user_ratings.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [11]:
movie_ratings_pivot = movie_user_ratings.pivot(index='movieId', columns='userId', values='rating')

In [12]:
movie_ratings_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
movie_ratings_pivot = movie_ratings_pivot.fillna(0)
movie_ratings_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
movie_ratings_pivot.shape

(9724, 610)

In [15]:
from scipy.sparse import csr_matrix
movie_ratings_matrix = csr_matrix(movie_ratings_pivot.values)

In [16]:
from sklearn.neighbors import NearestNeighbors
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)

In [17]:
knn_model.fit(movie_ratings_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=10, p=2, radius=1.0)

In [18]:
def get_id(movie_name):
    movie_id = movie_info.loc[movie_info['title']==movie_name]['movieId']
    return int(movie_id)

In [19]:
def get_title(index):
    movie_id = movie_ratings_pivot.index.values[index]
    movie_name = movie_info.loc[movie_info['movieId'] == movie_id]
    return movie_name['title'].values[0]

In [27]:
import numpy as np

def get_recommendations(movie_name, no_of_recommendations):
    movie_id = get_id(movie_name)
    matrix_index = np.where(movie_ratings_pivot.index.values == movie_id)[0][0]
    distances, indices = knn_model.kneighbors(movie_ratings_matrix[matrix_index], n_neighbors=no_of_recommendations)
    # zip distances and indices to create a list of tuples (index, distance)
    recommendations = list(zip(indices.squeeze().tolist(), distances.squeeze().tolist()))
    recommendations.pop(0)
    # now replace index with title
    rec_movies= [(get_title(item[0]), item[1]) for item in recommendations]
    for item in rec_movies:
        print(item)
get_recommendations('Avengers: Age of Ultron (2015)',10)

('Captain America: The Winter Soldier (2014)', 0.21896509582772627)
('Thor: The Dark World (2013)', 0.22208992683969697)
('Captain America: The First Avenger (2011)', 0.30635709393054333)
('X-Men: Days of Future Past (2014)', 0.3586709247729726)
('Thor (2011)', 0.35893484809667464)
('Avengers, The (2012)', 0.38340897847907185)
('Guardians of the Galaxy (2014)', 0.38493448620321546)
('Iron Man 2 (2010)', 0.39549306485723323)
('Iron Man 3 (2013)', 0.4132438056904373)


In [21]:
get_recommendations('Toy Story (1995)',10)

('Toy Story 2 (1999)', 0.4273987396802844)
('Jurassic Park (1993)', 0.4343631959138433)
('Independence Day (a.k.a. ID4) (1996)', 0.43573830647233425)
('Star Wars: Episode IV - A New Hope (1977)', 0.4426118294200634)
('Forrest Gump (1994)', 0.4529040920598262)
('Lion King, The (1994)', 0.4588546505397667)
('Star Wars: Episode VI - Return of the Jedi (1983)', 0.4589106952274158)
('Mission: Impossible (1996)', 0.46108722944164227)
('Groundhog Day (1993)', 0.465831237415656)


In [22]:
get_recommendations('Avengers: Age of Ultron (2015)',10)

('Captain America: The Winter Soldier (2014)', 0.21896509582772627)
('Thor: The Dark World (2013)', 0.22208992683969697)
('Captain America: The First Avenger (2011)', 0.30635709393054333)
('X-Men: Days of Future Past (2014)', 0.3586709247729726)
('Thor (2011)', 0.35893484809667464)
('Avengers, The (2012)', 0.38340897847907185)
('Guardians of the Galaxy (2014)', 0.38493448620321546)
('Iron Man 2 (2010)', 0.39549306485723323)
('Iron Man 3 (2013)', 0.4132438056904373)


In [23]:
get_recommendations('Jumanji (1995)',10)

('Lion King, The (1994)', 0.4115622741415874)
('Mrs. Doubtfire (1993)', 0.4501818938444998)
('Mask, The (1994)', 0.45501892320213067)
('Jurassic Park (1993)', 0.46195443302270334)
('Home Alone (1990)', 0.475123579391069)
('Nightmare Before Christmas, The (1993)', 0.48183868044092715)
('Aladdin (1992)', 0.484380023149225)
('Beauty and the Beast (1991)', 0.492542010867402)
('Ace Ventura: When Nature Calls (1995)', 0.5024397358631021)
